## Python | Questions

In [1]:
#!pip install autogen
#!python3 -m pip install pyautogen

In [2]:
import os
import autogen
import argparse
import sys
import logging
import json
import time

#### Get OpenAI API

In [4]:
gpt_model = "gpt-3.5-turbo"
model_tag = "3"

# gpt_model = "gpt-4-turbo"
# model_tag = "4"


llm_config = {"model": gpt_model,\
              "api_key": os.environ["OPENAI_API_KEY"],\
                "max_tokens": 2000}

### Question Builder

In [6]:
class Generator(object):
    
    def __init__(self, domain, title, detail, samples):
        self.domain = domain
        self.title = title
        self.detail = detail
        self.samples = str(samples)
        self.nitems = 10

    
    def create_writer_agent(self):
        question_writer_intro = "You write engaging and concise list of questions on given \
            interview topics and context.You will be provided some examples to get started. \
            You can use the examples to create more questions on the provided topics in machine learning. \
            Make sure you create questions that are clear and concise by following the pattern \
            of main question, explanation and follow-up questions. Create suggested number of questions."
        
        
        self.question_writer = autogen.AssistantAgent(
            name="Writer",
            description="Write engaging and concise list of questions on given interview topics and context.",
            system_message="You are a question creater for interview.\
            {question_writer_intro} Maximum word limit 1000",
            llm_config=llm_config)
        return
    

    def create_reviewer_agent(self):
        question_reviewer_intro = "You are a reviewer for the questions created by the question writer. \
            you dont reply anything except revised reply. Do not add any text like reply is great\
            You will be provided with a list of questions created by the question writer. \
            Your task is to review the questions and fix it if there is error and return revised one. \
            Make sure the questions are clear, concise and engaging."
        
        self.question_reviewer = autogen.AssistantAgent(
            name="Reviewer",
            description="Review the questions created by the question writer.",
            system_message="You are a reviewer for the questions created by the question writer.\
            {question_reviewer_intro} Maximum word limit 1000",
            llm_config=llm_config)
        return
    
    
    def build_task(self):
        task = f"""
            Create a list of questions for a {self.title} in {self.domain}.
            Use the provided {self.detail} to understand the domain, title and concept to be included in detail.
            The questions should be engaging and concise, following the format of the provided examples ({self.samples}).
            Each question must include:
            1. Main question
            2. Explanation
            3. Three follow-up questions

            Instructions:
            1. Collect all questions into a Python list, with each question as a dictionary 
            containing 'Main question', 'Explanation', and 'Follow-up questions' as keys.
            2. Ensure the value for each dictionary key is a proper string using the str() function.
            3. Follow-up questions should be in a list od three question strings.
            4. The final list should contain {self.nitems} question dictionaries.
            5. Write only the Python list, starting with [ and ending with ] as final contents.
            6. Do not include any text before or after the list.
            7. Avoid unauthorized text between the list of question dictionaries.
            8. Ensure it is a valid Python list, written in a readable key-value style, 
            organized in seperate line by line style.
            Must have: Avoid using words like "model's," "candidate's," and "interviewer's" 
            directly; use escape characters if needed (e.g., model\'s).
            Must have: make sure "'s" is escaped properly as "\'s" in the questions.
            Maximum word limit: 1000.
            """

        return task
    
    def review_task(self):
        task = f"""
            Review the list of questions created by the question writer for a {self.title} in {self.domain}.
            Use the provided {self.detail} to understand the domain, title and concept to be included in detail.
            Review and fix any errors in the format of the questions, rewriting sections if necessary.
            Refer to the provided examples ({self.samples}) for guidance.
            Suggested number of questions: {self.nitems}.

            Instructions:
            1. Ensure it is a collection of questions in a Python list, with each question as a dictionary containing
              'Main question', 'Explanation', and 'Follow-up questions' as keys.
            2. Ensure the value for each dictionary key is a proper string using the str() function if needed.
            3. Follow-up questions should be in a Python list including three question strings.
            4. The final list should contain {self.nitems} question dictionaries arranged into a Python list.
            5. Write only the Python list, starting with [ and ending with ] as final contents.
            6. Do not include any text before or after the Python list.
            7. Avoid unauthorized text between the list of question dictionaries.
            8. Ensure it is a valid Python list, written in a readable key-value style, organized in 
            seperate  line by line style.
            9. Avoid using words like "model's," "candidate's," and "interviewer's" directly; use escape
              characters if needed (e.g., model\'s).
            Must have: Avoid using words like "model's," "candidate's," and "interviewer's" directly; use
              escape characters if needed (e.g., model\'s).
            Must have: make sure "'s" is escaped properly as "\'s" in the questions
            Maximum word limit: 1000.
            """

        return task
    
    
    def generate_questions(self):

        # get task
        task = self.build_task()

        # generate questions
        reply = self.question_writer.generate_reply(messages=[{"content": task, "role": "user"}])

        # format the content of reply by extracting the python List only by 
        # removing "python```" at first and "```" at the end
        
        for line in reply.split("\n"):
            if line.startswith("```python"):
                reply = reply.replace(line, "")
            if line.startswith("```"):
                reply = reply.replace(line, "")
        
        # remove the text part that appears before list begins with "[" first time
        reply = reply[reply.index("["):]
        return reply
    

    def review_questions(self, reply):

        # get task
        pre_task = self.review_task()
        task = pre_task + f" here is the question list for you to review: {reply}"
        
        # generate questions
        reply = self.question_reviewer.generate_reply(messages=[{"content": task, "role": "user"}])

        # format the content of reply by extracting the python List only by 
        # removing "python```" at first and "```" at the end
        
        for line in reply.split("\n"):
            if line.startswith("```python"):
                reply = reply.replace(line, "")
            if line.startswith("```"):
                reply = reply.replace(line, "")
        
        # remove the text part that appears before list begins with "[" first time
        if reply.find("[") != -1:
            reply = reply[reply.index("["):]


        #use lowercase and remove space by _
        # replacr - by _, space by _ and make lower case in title
        self.title = self.title.replace(" ", "_").replace("-", "_").lower()

        file_name = "questions/" + self.title

        # save the questions to a python file
        with open(file_name + ".py", "w") as file:
            file.write(f"questions = {reply}")
            file.close()
        print(f"Questions are saved to {file_name}.py")
        return 

  

### Create Questions

In [8]:
with open("sectors/numpy.json", "r") as f:
    SECTORS = json.load(f)

In [9]:
QLIST = [item['titles'] for item in SECTORS]
QLIST

['Introduction to NumPy',
 'NumPy Installation',
 'Creating Arrays',
 'Array Attributes',
 'Array Indexing and Slicing',
 'Array Manipulation',
 'Element-wise Operations',
 'Broadcasting',
 'Mathematical Functions',
 'Statistical Functions',
 'Linear Algebra',
 'Random Number Generation',
 'Advanced Indexing',
 'Masked Arrays',
 'Structured Arrays',
 'Memory Management',
 'NumPy and C Extensions',
 'FFT (Fast Fourier Transform)',
 'Polynomials',
 'Performance Optimization',
 'Parallel Computing',
 'Integration with Pandas',
 'Integration with SciPy',
 'Saving and Loading Arrays',
 'Testing and Debugging',
 'Mathematical Constants']

In [10]:
from examples.example_questions import examples as example_questions

In [11]:
samples = str(example_questions)

for item in SECTORS:
    title = item["titles"]
    domain = item["domain"]
    detail = item["detail"]

    
    if title in QLIST:
        print(f"Generating questions for {title}")
        print("-------------------------")
        
        # prepare generate object
        generator = Generator(domain, title, detail, samples)

        # create agents
        generator.create_writer_agent()
        generator.create_reviewer_agent()

        # generate questions
        raw = generator.generate_questions()

        # review questions and save them to a file
        generator.review_questions(raw)

        time.sleep(5)

Generating questions for Introduction to NumPy
-------------------------
Questions are saved to questions/introduction_to_numpy.py
Generating questions for NumPy Installation
-------------------------
Questions are saved to questions/numpy_installation.py
Generating questions for Creating Arrays
-------------------------
Questions are saved to questions/creating_arrays.py
Generating questions for Array Attributes
-------------------------
Questions are saved to questions/array_attributes.py
Generating questions for Array Indexing and Slicing
-------------------------
Questions are saved to questions/array_indexing_and_slicing.py
Generating questions for Array Manipulation
-------------------------
Questions are saved to questions/array_manipulation.py
Generating questions for Element-wise Operations
-------------------------
Questions are saved to questions/element_wise_operations.py
Generating questions for Broadcasting
-------------------------
Questions are saved to questions/broadc